# https://openweathermap.org/api 에서 API 받기
위도, 경도 사용 <br>
ID: beegramin PW: 소1!

In [71]:
import requests
from urllib.parse import urlparse
import pandas as pd
from tqdm import tqdm

In [72]:
key_fd = open('../keys/openweatherkey.txt', mode='r')
# https://home.openweathermap.org/api_keys 에서 키받기
oweather_key = key_fd.read(100)
key_fd.close()

In [73]:
# https://openweathermap.org/current API 가져오기

# https://openweathermap.org/current#geo 키 집어넣는 방법, By geographic coordinates
# api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}

lat = 37.5509655144007
lng = 126.849532173376
units = 'metric'

url = f'http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lng}&appid={oweather_key}&units={units}&'

result = requests.get(urlparse(url).geturl()).json()
result

{'coord': {'lon': 126.85, 'lat': 37.55},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 3.06,
  'feels_like': -1.95,
  'temp_min': 2,
  'temp_max': 4,
  'pressure': 1019,
  'humidity': 74},
 'visibility': 10000,
 'wind': {'speed': 4.1, 'deg': 350},
 'clouds': {'all': 75},
 'dt': 1605846128,
 'sys': {'type': 1,
  'id': 8105,
  'country': 'KR',
  'sunrise': 1605824263,
  'sunset': 1605860344},
 'timezone': 32400,
 'id': 1838716,
 'name': 'Bucheon-si',
 'cod': 200}

In [74]:
weather = result['weather'][0]['main']
desc = result['weather'][0]['description']

weather,desc 

('Clouds', 'broken clouds')

In [75]:
# 3.05 반올림하면 3.0으로 나와서 약간 조절해줌
temp = round(result['main']['temp']+0.01,1)

temp_min = result['main']['temp_min']
temp_max = result['main']['temp_max']

temp, temp_min, temp_max

(3.1, 2, 4)

### address2.csv 파일에 온도 정보 추가하기

In [76]:
df = pd.read_csv('address2.csv', sep=',',encoding='utf8')
df

,구청,주소,lat,lng
0,강서구청,서울특별시 강서구 화곡로 302 (화곡동),37.550966,126.849532
1,양천구청,서울특별시 양천구 목동동로 105 (신정동),37.517075,126.866543
2,부천시청/부천시의회,경기도 부천시 길주로 210 (중동),37.503581,126.765337
3,영등포구청,서울특별시 영등포구 당산로 123 (당산동3가),37.525963,126.896367
4,김포시청,경기도 김포시 사우중로 1 (사우동),37.615190,126.715717


In [79]:
weather_list = []
desc_list = []
temp_list = []
temp_min_list = []
temp_max_list = []

In [81]:
for index in tqdm(df.index):
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={df['lat'][index]}&lon={df['lng'][index]}&appid={oweather_key}&units={units}"

    result = requests.get(urlparse(url).geturl()).json()
    
    weather_list.append(result['weather'][0]['main'])
    desc_list.append(result['weather'][0]['description'])
    temp_list.append(round(result['main']['temp']+0.01,1))
    temp_min_list.append(result['main']['temp_min'])
    temp_max_list.append(result['main']['temp_max'])

100%|██████████| 5/5 [00:02<00:00,  1.76it/s]


In [83]:
df['weather'] = weather_list
df['desc'] = desc_list
df['temp'] = temp_list
df['temp_min'] = temp_min_list
df['temp_max'] = temp_max_list

In [84]:
df

,구청,주소,lat,lng,weather,desc,temp,temp_min,temp_max
0,강서구청,서울특별시 강서구 화곡로 302 (화곡동),37.550966,126.849532,Clouds,broken clouds,3.1,2,4
1,양천구청,서울특별시 양천구 목동동로 105 (신정동),37.517075,126.866543,Clouds,broken clouds,4.3,4,5
2,부천시청/부천시의회,경기도 부천시 길주로 210 (중동),37.503581,126.765337,Clouds,broken clouds,3.0,2,4
3,영등포구청,서울특별시 영등포구 당산로 123 (당산동3가),37.525963,126.896367,Clouds,overcast clouds,3.9,3,5
4,김포시청,경기도 김포시 사우중로 1 (사우동),37.615190,126.715717,Clouds,broken clouds,3.4,2,4
